In [2]:
import os
import pickle
import plotting
import custom_tools
import scanpy as sc
import numpy as np
from tqdm import tqdm
from dataset import TissueDataset
import matplotlib.pyplot as plt

dataset_name = "JacksonFischer"
RAW_DATA_PATH = os.path.join("../data", f"{dataset_name}/raw")
dataset = TissueDataset(os.path.join(f"../data/{dataset_name}", "month"),  "month")

adata_exp = sc.read_h5ad(f"../data/out_data/adatafiles/JacksonFischer/JacksonFischer_JF_concatenated_explanations.h5ad")
adata_exp.obs_names_make_unique()

/home/rifaioglu/miniconda3/envs/py_gem/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42


Target prediction: OSmonth


/home/rifaioglu/miniconda3/envs/py_gem/lib/python3.11/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [10]:
import pandas as pd
from scipy.stats import chi2_contingency, fisher_exact
count=0
for test_graph in tqdm(dataset):
    gene_list = custom_tools.get_gene_list()
    with open(os.path.join(RAW_DATA_PATH, f'{test_graph.img_id}_{test_graph.p_id}_coordinates.pickle'), 'rb') as handle:
                coordinates_arr = pickle.load(handle)
    tmp_adata_exp = adata_exp[adata_exp.obs.img_id==test_graph.img_id,:]
    
    df = tmp_adata_exp.obs.copy()
    
    # Convert array to DataFrame
    new_columns = pd.DataFrame(test_graph.x, columns=gene_list)
    
    # Reset both indices to avoid index alignment issues
    df = df.reset_index(drop=True)
    new_columns = new_columns.reset_index(drop=True)

    # Concatenate without considering index values
    df = pd.concat([df, new_columns], axis=1)

    # Subset only important regions
    important_df = df[df["importance_hard"] == "True"]
    # Compute the fraction of important regions that are Tumor
    tumor_fraction_in_important = (important_df["class"] == "Tumor").mean()

    print(f"Fraction of important regions that are Tumor: {tumor_fraction_in_important:.2f}")
    
    if tumor_fraction_in_important>0.7:
        node_to_score_dict = dict()
        node_to_imp_dict = dict()
        node_to_gene_dict = dict()
        
        for node_idx, row in df.iterrows():
            # node_idx =  int(ind.split("-")[0])
            node_to_score_dict[node_idx] = row.importance
            node_to_gene_dict[node_idx]  = row["panCK"]
            node_to_imp_dict[node_idx] = 1 if row.importance_hard=="True" else 0
            
        print(important_df)
        
        img_id = test_graph.img_id
        plt.rcParams['figure.figsize'] = 48, 8
        fig, axs = plt.subplots(1, 4)
        # plotting.plot_graph(test_graph, coordinates_arr, axs[0], font_size=5,  node_size=100, width=1)
        plotting.plot_node_importances_voronoi(test_graph, coordinates_arr, node_to_imp_dict,  axs[0], cmap=plt.cm.afmhot_r)
        # plotting.plot_node_importances(test_graph, coordinates_arr, node_to_score_dict,  axs[2], node_size=100, width=1)
        plotting.plot_node_importances_voronoi(test_graph, coordinates_arr, node_to_score_dict, axs[1])
        plotting.plot_node_importances_voronoi(test_graph, coordinates_arr, node_to_gene_dict, axs[2])
        plotting.plot_node_types_voronoi(test_graph, coordinates_arr, axs[3])
        plt.savefig(f"/home/rifaioglu/projects/GNNClinicalOutcomePrediction/plots/explanations/JacksonFischer/{test_graph.img_id}.pdf")
        plt.savefig(f"/home/rifaioglu/projects/GNNClinicalOutcomePrediction/plots/explanations/JacksonFischer/{test_graph.img_id}.png")
        plt.clf()

    count +=1
    if count==10:
        break
    

  0%|          | 0/1114 [00:00<?, ?it/s]

Fraction of important regions that are Tumor: 0.38
Fraction of important regions that are Tumor: 0.74
    clinical_type img_id      p_id  tumor_grade  osmonth  importance  \
1       TripleNeg  183lr  81-basel  tensor([3])     89.0    0.335272   
18      TripleNeg  183lr  81-basel  tensor([3])     89.0    0.262929   
30      TripleNeg  183lr  81-basel  tensor([3])     89.0    0.303257   
32      TripleNeg  183lr  81-basel  tensor([3])     89.0    0.298782   
35      TripleNeg  183lr  81-basel  tensor([3])     89.0    0.275203   
..            ...    ...       ...          ...      ...         ...   
702     TripleNeg  183lr  81-basel  tensor([3])     89.0    0.254088   
715     TripleNeg  183lr  81-basel  tensor([3])     89.0    0.262406   
730     TripleNeg  183lr  81-basel  tensor([3])     89.0    0.264425   
740     TripleNeg  183lr  81-basel  tensor([3])     89.0    0.254104   
747     TripleNeg  183lr  81-basel  tensor([3])     89.0    0.256631   

    importance_hard        cell_t

  0%|          | 2/1114 [00:02<26:22,  1.42s/it]

Fraction of important regions that are Tumor: 0.27
Fraction of important regions that are Tumor: 0.64
Fraction of important regions that are Tumor: 0.78
    clinical_type img_id       p_id  tumor_grade  osmonth  importance  \
5        HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.263294   
6        HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.273113   
7        HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.279054   
8        HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.220616   
12       HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.219089   
14       HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.225291   
15       HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.228822   
18       HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.228927   
21       HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.293923   
30       HR+HER2-  231lr  258-basel  tensor([2])     80.0    0.212549   
33       HR+HER2-  231lr  258-basel  tensor(

  0%|          | 5/1114 [00:05<17:45,  1.04it/s]

...

  1%|          | 9/1114 [00:07<14:35,  1.26it/s]

Fraction of important regions that are Tumor: 0.67
Fraction of important regions that are Tumor: 0.19


<Figure size 4800x800 with 0 Axes>

<Figure size 4800x800 with 0 Axes>

<Figure size 4800x800 with 0 Axes>

Contingency Table:
importance_hard  False  True 
class                        
False              756     90
True               236     21
Chi-square test p-value: 0.3016160221087665
Fraction of Tumor cells that are important: 0.08
Fraction of Non-Tumor cells that are important: 0.11


In [24]:
df["importance_hard"]

0-580      False
1-580      False
2-580      False
3-580      False
4-580      False
           ...  
1098-76    False
1099-76    False
1100-76     True
1101-76    False
1102-74    False
Name: importance_hard, Length: 1103, dtype: category
Categories (2, object): ['False', 'True']

In [35]:
test_graph.x.shape

torch.Size([1319, 33])

In [34]:
df

,clinical_type,img_id,p_id,tumor_grade,osmonth,importance,importance_hard,cell_type,class,H3K27me3,...,E/P-cadherin,Ki-67,EGFR,pS6,vWF/CD31,pmTOR,CK7,panCK,CC3/cPARP,HH3
0-338,HR+HER2-,147,253-basel,tensor([2]),112.0,0.135877,False,Small elongated,Stroma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-338,HR+HER2-,147,253-basel,tensor([2]),112.0,0.121110,False,Myoepithelial,Tumor,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2-338,HR+HER2-,147,253-basel,tensor([2]),112.0,0.132763,False,Myoepithelial,Tumor,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3-338,HR+HER2-,147,253-basel,tensor([2]),112.0,0.135291,False,Myoepithelial,Tumor,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4-338,HR+HER2-,147,253-basel,tensor([2]),112.0,0.118582,False,CK+ HR lo,Tumor,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.182822,...,0.219566,0.027411,0.139736,1.166651,0.000000,0.190310,0.011566,0.015504,0.279721,4.933357
1315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.368286,...,0.261821,0.008929,0.151438,1.428473,0.013554,0.252786,0.000000,0.008929,0.165402,1.065938
1316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.344820,...,0.301126,0.032054,0.241689,1.981605,0.070126,0.257862,0.025078,0.030479,0.332377,2.846970
1317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.173973,...,0.207541,0.000000,0.000000,0.573068,0.000000,0.144203,0.013892,0.013514,0.140419,1.380176
